In [ ]:
import asyncio
import json
import requests
from datetime import datetime, timezone
from env.utility.helps import Bob
from env.utility.file_management import File_Management
from env.audit import Audits
from croniter import croniter

In [ ]:
from env.modules.activity import main as Activity
from env.modules.apps import main as Apps
from env.modules.catalog import main as Catalog
from env.modules.graph import main as Graph
from env.modules.tenant import main as Tenant
from env.modules.refreshhistory import main as RefreshHistory
from env.modules.refreshables import main as Refreshables
from env.modules.gateway import main as Gateway
from env.modules.capacity import main as Capacity

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

keyvault_url = "https://powerbimonitorkv.vault.azure.net/"

# Create an instance of DefaultAzureCredential
credential = DefaultAzureCredential()

# Create a SecretClient instance
client = SecretClient(vault_url=keyvault_url, credential=credential)

# Use the client to access secrets in Key Vault
# keyvault_url = "https://powerbimonitorkv.vault.azure.net/"

# from notebook.utils import mssparkutils


In [ ]:
print("hello world...I updated my repo")

In [ ]:
import os
from env.audit import Audits

appsecret = os.getenv("admin_appsecret")
appid = os.getenv("admin_appid")
tenantid = os.getenv("admin_tenantid")

audit = Audits()
audit.set_ServicePrincipal(
    tenant_id=tenantid,
    client_id=appid,
    client_secret= appsecret
)

# audit.set_ApplicationModules("Activity,Apps,Capacity,Catalog,Domains,FabricItems,Gateway,Graph,Refreshables,RefreshHistory,Roles,Tenant,Workspaces")
audit.set_ApplicationModules("Activity")

audit.set_LakehouseName("FabricLake")
audit.set_PathInLakehouse("stage")
audit.set_WorkspaceName("FabricMonitor")

audit.set_Activity_cron("* * * * * 30")

# print(audit.ServicePrincipal)
await audit.run()



In [ ]:
audit.set_Catalog_cron("* * * * * 30")
audit.set_Capacity_cron("* * * * * 30")
audit.set_Apps_cron("* * * * * 30")
audit.set_Domains_cron("* * * * * 30") # Not yet implemented
audit.set_Graph_cron("* * * * * 30")
audit.set_Tenant_cron("* * * * * 30")
audit.set_RefreshHistory_cron("* * * * * 30")
audit.set_Refreshables_cron("* * * * * 30")
audit.set_Gateway_cron("* * * * * 30")



In [ ]:
t = '{"Domains":{"lastRun":"today"}}'
import json
j = json.loads(t)
print(j)

j = t.get("Domainss",{}).get("lastRun")
if j:
    print("found")
else:
    print("not found")

In [ ]:
from env.audit import Audits

audit = Audits()  # Await the coroutine object
audit.LakehouseName = "FabricLakehouse"

# audit.ServicePrincipal["AppId"] = "a4f7b4d5-7d0e-4b0b-a5b1-4e2b8c8f6e2e"
from env.utility.helps import Bob
bob = Bob()
bob.audit = audit

bob.audit.LakehouseName = "FabricLakeHome"

print(bob.audit.LakehouseName)

In [ ]:


def scheduler(cron_time, last_run=None):

    year = datetime.now().year
    month = datetime.now().month
    day = datetime.now().day

    local_date = datetime(year, month, day, tzinfo=timezone.utc)
    val = croniter(cron_time, local_date).get_next(datetime)
    last_run = bob.convert_dt_str(last_run)
    print(f"did i even know this value {last_run}")

    # Determine if the current day of the week is the same as the value for day of the week in the cron value
    current_day_of_week = datetime.now().strftime("%A")
    cron_day_of_week = cron_time.split(" ")[4]


    # Get the next scheduled datetime
    next_datetime = val
    print(next_datetime)

    # Check if the current datetime matches the next scheduled datetime
    if datetime.now().strftime("%Y-%m-%d") == next_datetime.strftime("%Y-%m-%d"):
        print("Hello World, I am supposed to run today")
    else:
        print("Goodnight World, I am sleeping")


In [ ]:
def is_function_due(cron_syntax, last_run):
    # last_run_datetime = last_run
    now = datetime.now()
    cron = croniter(cron_syntax, now)
    
    next_run_datetime = cron.get_next(datetime)

    if next_run_datetime.strftime("%Y-%m-%d") <= datetime.now().strftime("%Y-%m-%d"):
        return {"IsDue": True, "NextRun": next_run_datetime, "LastRun": last_run}
    else:
        return {"IsDue": False, "NextRun": next_run_datetime, "LastRun": last_run}



In [ ]:
def test_cron(sched='0 4 * * 6'):
    now = datetime.now()
    cron = croniter(sched, now)

    for i in range(8):
        nextdate = cron.get_next(datetime)
        print (nextdate)



In [ ]:
test_cron("0 0 * * 0-6")

In [ ]:
from datetime import datetime

modules = settings.get("ApplicationModules").replace(" ","").split(",")
# classes = [globals()[module] for module in modules]

classes = ["Activity"]


for module in classes:
    sched = settings.get(f"{module}_cron")
    run = state.get(f"{module.lower()}").get("lastRun")
    print(sched)
    last_run = datetime.now()#  bob.convert_dt_str(run)    

    results = is_function_due(sched,last_run)
    r = test_cron(sched)

    if results.get("IsDue"):
        print(f"Function {module} last ran on {run} and is due to run at {results.get('NextRun')} compared to {r}")
    else:
        print(f"Function {module} last ran on {run} and is not due to run but will run after {results.get('NextRun')} compared to {r}")




In [ ]:
settings.get("Refreshables_cron")
state.get("refreshables").get("lastRun")

In [ ]:
state.get('activity').get('lastRun')

In [ ]:
api_url = 'https://api.powerbi.com/v1.0/myorg/gateways'
response = requests.get(url=api_url, headers=headers)
results = response.json()
print(response.status_code)
gateways = list()
for gateway in results['value']:
    gateways.append(gateway.get("id"))


In [ ]:
r = list()
for id in gateways:
    response = requests.get(f'https://api.powerbi.com/v1.0/myorg/gateways/{id}/datasources', headers=headers)
    doc_results = response.json()
    r.append(doc_results['value'])   


In [ ]:
gateways = list()

for i in range(0, len(r)):
    if isinstance(r[i], list):
        for j in range(0, len(r[i])):
            gateways.append(r[i][j])
    else:
        gateways.append(r[i])

In [ ]:
gateways

In [ ]:
users = list()

for i in range(0,len(gateways)):
    api_users = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/users'
    response = requests.get(api_users, headers=headers)
    results = response.json()
    results['value'][0]['datasourceId'] = gateways[i]["id"]
    results['value'][0]['gatewayId'] = gateways[i]["gatewayId"]
    users.append(results['value'][0])
    

## Datasource Connectivity  

This only reports back connectivity to the datasource as an error. Also, you need to use the response.text as the JSON is not populated. A successful connection will report back as HTTP 200 while a failed connection will result in HTTP 400

In [ ]:
## This will be used for a template dictionary for a successful connection to the datasource
status_base = {   
    "datasourceId":"",
    "gatewayId":"",
    "error":
    {
        "code":"success",
        "pbi.error":{ 
            "code":"0",
            "parameters":{},
            "details":[],
            "exceptionCulprit":0
        }
    }
}


In [ ]:
status  = list()
for i in range(0,len(gateways)):
    api_status = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}/status'
    
    response = requests.get(api_status, headers=headers)
    if response.ok:
        # response.json() is actually empty when it is a success
        
        status_base['datasourceId'] = gateways[i]["id"]
        status_base['gatewayId'] = gateways[i]["gatewayId"]
        status_base.get("error").get("pbi.error").get("details").append({"message":"success","detail":response.status_code})    
        status.append(status_base)
    else:
        results = json.loads(response.text)
        results["datasourceId"] = gateways[i]["id"]
        results["gatewayId"] = gateways[i]["gatewayId"]
        results.get("error").get("pbi.error").get("details").append({"message":"cannot connect","detail":response.status_code})    
        status.append(results)


In [ ]:
datasources = list()

for i in range(0,len(gateways)):
    api_datasource = f'https://api.powerbi.com/v1.0/myorg/gateways/{gateways[i]["gatewayId"]}/datasources/{gateways[i]["id"]}'
    response = requests.get(api_datasource, headers=headers)
    results = response.json()
    results.pop('@odata.context')
    datasources.append(results)


In [ ]:
import smtplib
from email.mime.text import MIMEText

sender_email = 'brcampb@microsoft.com'
receiver_email = 'brandonh.campbell@gmail.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Gmail)
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())



sender_email = 'your_email@example.com'
receiver_email = 'recipient_email@example.com'
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'

msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = receiver_email

# Set up the SMTP server (e.g., Outlook)
with smtplib.SMTP('smtp.office365.com', 587) as server:
    server.starttls()
    server.login(sender_email, 'your_password')
    server.sendmail(sender_email, [receiver_email], msg.as_string())


In [ ]:
import smtplib
from email.mime.text import MIMEText

# Set up the SMTP server
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'your_email@example.com'
password = 'your_password'

# Create a message
subject = 'Hello from Python!'
body = 'This is a test email sent from a Python script.'
msg = MIMEText(body)
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = 'recipient_email@example.com'

# Send the email
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, [msg['To']], msg.as_string())


In [ ]:
# rest_api = "admin/capacities/refreshables"
# GET https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
# htpps://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets
rest_api = "https://api.powerbi.com/v1.0/myorg/admin/groups?$expand=datasets&$top=5000"

response = requests.get(url=rest_api, headers=headers)
workspaces = response.json()


In [ ]:
workspaces

In [ ]:
for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        print(dataset)

In [ ]:
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/datasets/{datasetId}/refreshes
refresh_history = list()

for item in workspaces['value']:
    group_id = item['id']
    for dataset in item['datasets']:
        dataset_id = dataset['id']
        if dataset['isRefreshable']==True and dataset['addRowsAPIEnabled']==False:
            rest_api = f"https://api.powerbi.com/v1.0/myorg/groups/{group_id}/datasets/{dataset_id}/refreshes"

            response = requests.get(url=rest_api, headers=headers)
            if response.ok:
                results = response.json()
                for result in results['value']:
                    if len(result)>0:
                        refresh_history.append(result)




In [ ]:
refresh_history

In [ ]:
import pandas as pd

fd = pd.DataFrame(refresh_history)
fd.head()

In [ ]:
fd[fd['requestId']=='9cbca636-ec64-4ff6-9dfe-9e2e5acc3056']['refreshAttempts'][0]

In [ ]:
import pandas as pd

def normalize(df:pd.DataFrame(), base2:pd.DataFrame(), json_column='workspace_id'):
    for row in df.itertuples():
        column_number = df.columns.get_loc(json_column)+1
        if isinstance(row[column_number], list) and len(row[column_number])>0:
            for item in row[column_number]:
                # item[id_name] = row.id
                base2 = pd.concat([base2, df, pd.json_normalize(item)])
                base2.drop(json_column, axis=1, inplace=True)
        else:
            if isinstance(row[column_number], dict):
                print("Yes it is a dictionary")
                base2 = pd.concat([base2, df, pd.json_normalize(row[column_number])])
                base2.drop(json_column, axis=1, inplace=True)
    return base2

In [ ]:
dd = normalize(fd, pd.DataFrame(), 'refreshAttempts')
dd.head()

In [ ]:
de = normalize(dd, pd.DataFrame(), 'serviceExceptionJson')
de

In [ ]:
import json

string_dict = "{'id': '8271b9f0-3ff7-46b5-9f8a-5a3ba59d01b7', 'name': 'FabricMonitor', 'addRowsAPIEnabled': False, 'configuredBy': 'brandon.campbell@mngenvmcap084084.onmicrosoft.com', 'isRefreshable': True, 'isEffectiveIdentityRequired': False, 'isEffectiveIdentityRolesRequired': False, 'targetStorageMode': 'Abf', 'createdDate': '2024-03-27T18:42:49.663Z', 'contentProviderType': 'PbixInImportMode', 'upstreamDatasets': [], 'users': [], 'isInPlaceSharingEnabled': False}"



print(dict_obj)


In [ ]:
for row in dd.itertuples():
    if isinstance(row.serviceExceptionJson, list) and len(row.serviceExceptionJson)>0:
        for item in row.serviceExceptionJson:
            print(item)
    else:
        if isinstance(row.serviceExceptionJson, str):
            print("Yes it is a dictionary")
            print(row.serviceExceptionJson)

In [ ]:
api="https://api.powerbi.com/v1.0/myorg/admin/capacities/refreshables?$top=5000"

refreshables = list()

response = requests.get(url=api, headers=headers)
if response.ok:
    results = response.json()
    for result in results['value']:
        refreshables.append(result)

        

In [ ]:
if len(refresh_history)>=len(refreshables):
    print("there is more information in refresh_history than in refreshables")
else:
    print("there is more information in refreshables than in refresh_history")

In [ ]:
refresh_history[0]

In [ ]:
import pandas as pd

df = pd.json_normalize(refreshables)
df.head()

In [ ]:
df2 = pd.json_normalize(refresh_history)

In [ ]:
df2.head()

In [ ]:
df.to_csv('refreshables.csv', index=False)
df2.to_csv('refresh_history.csv', index=False)

In [ ]:
df3 = df2[['requestId','refreshAttempts']]

In [ ]:
import pandas as pd

# Assuming the DataFrame with the column 'refreshAttempts' is named 'df'
df_expanded = pd.json_normalize(df3['refreshAttempts'])

# Merge the expanded DataFrame with the original DataFrame
df_merged = pd.concat([df3.drop('refreshAttempts', axis=1), df_expanded], axis=1)


In [ ]:
df_merged.head()

In [ ]:
df_unpivoted = df_merged.melt(id_vars=['requestId'], var_name='column', value_name='value')
df_unpivoted.rename(columns={'requestId':'requestId','column':'column','value':'refreshAttempts'}, inplace=True)



In [ ]:
df_unpivoted.head()

In [ ]:
import pandas as pd
import json

# Convert JSON column to DataFrame
df_refreshAttempts = pd.json_normalize(df_unpivoted['refreshAttempts'])

# Merge the original DataFrame with the new DataFrame
df_merged = pd.concat([df_unpivoted.drop('refreshAttempts', axis=1), df_refreshAttempts], axis=1)
df_merged.head()

In [ ]:
refresh_history_clean = df_merged[['requestId','attemptId','startTime','endTime','serviceExceptionJson']]
refresh_history_clean.head()

In [ ]:
refresh_history_cleanest = df2.merge(refresh_history_clean, on='requestId', how='left')
refresh_history_cleanest.head()

In [ ]:
refresh_history_cleanest.drop(columns=['refreshAttempts'], inplace=True)

In [ ]:
refresh_history_cleanest.to_csv('refresh_history_cleanest.csv', index=False)

In [ ]:
args = ['--base','true','--fullscan','30']
for i in range(0, len(args), 2):
    print(args[i])
    # if args[i] == '--base':
    #     print(args[i+1])


In [ ]:
response = requests.get("https://api.fabric.microsoft.com/v1/admin/domains", headers=headers)
if response.ok:
    print(response.json())
    v = response.json()


In [ ]:
v

In [ ]:
domains = v.get("domains")
for domain in domains:
    domainId = domain['id']

    response = requests.get(f"https://api.fabric.microsoft.com/v1/admin/domains/{domainId}/workspaces", headers=headers)
    if response.ok:
        print(response.json())
    else:
        print(response.status_code)

In [ ]:
url = "https://api.powerbi.com/v1.0/myorg/admin/capacities"
response = requests.get(url, headers=headers)
if response.ok:
    print(response)

In [ ]:
response = requests.get("https://api.fabric.microsoft.com/v1/admin/workspaces", headers=headers)
if response.ok:
    results = response.json()

workspace = results.get("workspaces")

items = list()

for item in workspace:
    items.append(item["id"])



In [ ]:
items

In [ ]:



url = f"https://api.powerbi.com/v1.0/myorg/admin/groups/ef61e238-9402-4a64-9107-1e10f5d0fcc4?$expand=users"

response = requests.get(url, headers=headers)
if response.ok:
    results = response.json()
    print(results)
    


In [ ]:

items = set(items)
workspace_lst = list()

ceiling = len(items)
print(ceiling)
cnt = 0
for item in items:
    cnt+=1

    if len(workspace_lst) > ceiling:
        break

    if cnt <= ceiling:
        url = f"https://api.powerbi.com/v1.0/myorg/admin/groups/{item}?$expand=users"
    
        response = requests.get(url, headers=headers)
        if response.ok:
            results = response.json()
            workspace_lst.append(results)
        else:
            if response.status_code==429:
                result = response.json()
                print(f"you must wait { int(result.get('message').split('.')[1].split(' ')[3])/60} minutes")

                break

        

In [ ]:
# del(workspace_lst)

In [ ]:
import pandas as pd
import json

df = pd.read_json(json.dumps(workspace_lst))

In [ ]:
df.head()

In [ ]:
response = {"message": "You have exceeded the amount of requests allowed in the current time frame and further requests will fail. Retry in 1223 seconds."}
message = response.get("message")

tm = message.split(sep=".")[1].split(sep=" ")[3]

print(tm)

In [ ]:
import concurrent.futures

# Sample list of 50000 workspace IDs
workspace_ids = list(range(1, 50001))

# Function to process a single group of workspace IDs
def process_group(group):
    # Placeholder function to process a group
    print(f"Processing group with {len(group)} workspace IDs")

# Split the list into groups of 100 workspace IDs
groups = [workspace_ids[i:i + 100] for i in range(0, len(workspace_ids), 100)]

# Function to process groups in batches of 16
def process_batches(groups):
    batch_size = 16
    while groups:
        # Take the first 16 groups (or fewer if less than 16 remain)
        batch = groups[:batch_size]
        # Process the batch in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(process_group, batch)
        # Remove the processed groups from the list
        groups = groups[batch_size:]
        # Check if the number of remaining groups is less than 16
        if len(groups) < batch_size:
            print("Adding next group to be tested")

# Process the groups in batches
process_batches(groups)

# This script does the following:
# 1. Creates a sample list of 50000 workspace IDs.
# 2. Defines a `process_group` function to process a single group of workspace IDs.
# 3. Splits the list into groups of 100 workspace IDs.
# 4. Defines a `process_batches` function to process the groups in batches of 16, running them in parallel using `ThreadPoolExecutor` and removing each group from the batch once processed.

# Feel free to modify the `process_group` function to suit your specific needs. Let me know if you have any questions or need further assistance!

In [ ]:
# Got it! Here's a Python script that will help you achieve this. The script will chunk the list of 5000 workspace IDs into groups of 100, process them in parallel with a maximum of 16 concurrent functions, and add new functions as soon as any of the 16 completes.

# ```python
import concurrent.futures
import time

# Sample list of 5000 workspace IDs
workspace_ids = list(range(1, 50001))

# Function to process a single group of workspace IDs
def process_group(group):
    # Placeholder function to process a group
    print(f"Processing group with {len(group)} workspace IDs")
    for workspace_id in group:
        # Placeholder processing for each workspace ID
        print(f"Processing workspace ID: {workspace_id}")
    print("Sleeping for 5 seconds")
    time.sleep(5)  # Simulate some processing time

# Split the list into groups of 100 workspace IDs
groups = [workspace_ids[i:i + 100] for i in range(0, len(workspace_ids), 100)]

# Function to process groups in parallel with a maximum of 16 concurrent functions
def process_batches(groups):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = {executor.submit(process_group, group): group for group in groups[:16]}
        remaining_groups = groups[16:]

        while futures:
            # Wait for the next function to complete
            done, _ = concurrent.futures.wait(futures, return_when=concurrent.futures.FIRST_COMPLETED)

            for future in done:
                futures.pop(future)
                if remaining_groups:
                    next_group = remaining_groups.pop(0)
                    futures[executor.submit(process_group, next_group)] = next_group

# Process the groups in batches
process_batches(groups)
# ```

# This script does the following:
# 1. Creates a sample list of 5000 workspace IDs.
# 2. Defines a `process_group` function to process a single group of workspace IDs.
# 3. Splits the list into groups of 100 workspace IDs.
# 4. Defines a `process_batches` function to process the groups in parallel with a maximum of 16 concurrent functions using `ThreadPoolExecutor`. It adds new functions as soon as any of the 16 completes.

# Feel free to modify the `process_group` function to suit your specific needs. Let me know if you have any questions or need further assistance!

In [6]:
import asyncio

# Sample list of 5000 workspace IDs
workspace_ids = list(range(1, 5001))

# Async function to process a single group of workspace IDs
async def process_group(group):
    # Placeholder function to process a group
    print(f"Processing group with {len(group)} workspace IDs")
    await asyncio.sleep(1)  # Simulate some processing time

# Split the list into groups of 100 workspace IDs
groups = [workspace_ids[i:i + 100] for i in range(0, len(workspace_ids), 100)]

# Function to process groups in parallel with a maximum of 16 concurrent functions
async def process_batches(groups):
    semaphore = asyncio.Semaphore(16)

    async def process_with_semaphore(group):
        async with semaphore:
            await process_group(group)

    tasks = [process_with_semaphore(group) for group in groups]
    await asyncio.gather(*tasks)


In [ ]:

# To achieve this, you can use Python's `asyncio` and `concurrent.futures` modules to chunk your list of 50,000 workspace IDs into groups of 100 and process them using an async function with a maximum of 16 concurrent threads. Here's a step-by-step guide to help you:

# 1. **Chunk the list into groups of 100**: You can use a simple function to split your list into smaller chunks.
# 2. **Create an async function to process each chunk**: This function will handle the processing of each group of workspace IDs.
# 3. **Use a semaphore to limit the number of concurrent async tasks**: This will ensure that no more than 16 async tasks are running at the same time.
# 4. **Run the async tasks using `asyncio`**: Use the event loop to manage the execution of the async tasks.

# Here's an example to illustrate this:

# ```python
import asyncio
from concurrent.futures import ThreadPoolExecutor

# Function to chunk the list into groups of 100
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

# Async function to process each chunk
async def process_chunk(chunk):
    await asyncio.sleep(1)  # Simulate async processing
    return f"Processed {len(chunk)} workspace IDs"

# Function to run async tasks with a semaphore
async def run_async_tasks(workspace_ids, max_concurrent_tasks):
    semaphore = asyncio.Semaphore(max_concurrent_tasks)
    
    async def sem_task(chunk):
        async with semaphore:
            return await process_chunk(chunk)
    
    tasks = [sem_task(chunk) for chunk in chunk_list(workspace_ids, 100)]
    results = await asyncio.gather(*tasks)
    return results

# Main function to execute the async tasks
def main():
    workspace_ids = list(range(50000))  # Example list of 50,000 workspace IDs
    max_concurrent_tasks = 16
    
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(run_async_tasks(workspace_ids, max_concurrent_tasks))
    loop.close()
    
    for result in results:
        print(result)


main()
# ```

# In this example:
# - The `chunk_list` function splits the list of workspace IDs into chunks of 100.
# - The `process_chunk` async function simulates processing each chunk.
# - The `run_async_tasks` function uses a semaphore to limit the number of concurrent async tasks to 16.
# - The `main` function sets up the event loop and runs the async tasks.

# This approach ensures that your async function processes the workspace IDs in chunks of 100, with a maximum of 16 concurrent threads, without exceeding the limit. If you have any specific requirements or encounter any issues, feel free to let me know!

In [ ]:
d = {"name":"Brandon","age":30}
d.keys()

In [ ]:
import requests

url = "https://api.powerbi.com/v1.0/myorg/admin/capacities/refreshables"
response = requests.get(url, headers=headers, json=json)

In [ ]:
if "name" in d:
    print("yes")

In [ ]:
import time
group_size = 100
input_list = list(range(1, 50001))

groups = [input_list[i:i + group_size] for i in range(0, len(input_list), group_size)]

def process_items(items):
    counter = int()
    for i in range(0, len(items), 16):
        batch = items[i:i+16]

        for j in batch:
            counter+=1
            print(f"What row am I on {counter} and Processing item: {j}")
        # Process the batch
        # print(f"Processing batch: {batch} and length is {len(batch)}")
        time.sleep(2)  # Simulate processing time for each batch

        if i % 16 == 0:
            print("Pausing for 10 minutes...")
            time.sleep(2)  # Pause for 10 minutes

# Example usage

process_items(groups)


In [ ]:
len(groups[1])

In [ ]:
((532/16) * 30)/60
